In [1]:
import util
from agent_v2 import Agent
from mining_planner import MiningPlanner, MiningRecommendation, MiningRoutes

In [2]:
test_env = util.get_test_env()
test_env

In [3]:
agent = Agent('player_0', test_env.env_cfg)
agent

In [4]:
%%time
agent.update_step_info(test_env.state.env_steps, test_env.state.get_obs(), 60)

CPU times: total: 0 ns
Wall time: 3 ms


In [5]:
%%time
mining_planner = MiningPlanner(agent.master_plan_)

AttributeError: 'Agent' object has no attribute 'master_plan_'

In [6]:
%%time
routes = mining_planner._generate_routes()

NameError: name 'mining_planner' is not defined

In [7]:
# pathfinder = mining_planner.master_plan.pathfinder
from pathfinding.core.grid import Grid
from pathfinding.finder.a_star import AStarFinder

In [8]:
start_coord = [10, 11]
end_coord = [14, 15]
map_cost = test_env.state.board.rubble + 10

In [9]:
%%time
grid = Grid(matrix=map_cost)
start = grid.node(*start_coord)
end = grid.node(*end_coord)

CPU times: total: 15.6 ms
Wall time: 7.99 ms


In [10]:
%%time
pathfinder = AStarFinder(diagonal_movement=-1)

CPU times: total: 0 ns
Wall time: 0 ns


In [11]:
%%time
path1, runs = pathfinder.find_path(start, end, grid)

CPU times: total: 15.6 ms
Wall time: 2.99 ms


In [18]:
path4 == path1

True

In [13]:
print(f'Paths equal {path1 == path2} <<<< i.e. cannot reuse pathfinder more than once without grid.')

Paths equal False <<<< i.e. cannot reuse pathfinder more than once without grid.


In [14]:
%%time
grid.cleanup()
path3, runs = pathfinder.find_path(start, end, grid)

CPU times: total: 15.6 ms
Wall time: 4 ms


In [15]:
print(f'Paths equal {path1 == path3}')

Paths equal True


## Speed up pathfinding
Search smaller area

First by setting -1 everywhere outside of small area (that would keep dims the same etc)

In [28]:
import numpy as np
# start_coord = [10, 10]
# end_coord = [17, 17]
# map_cost = test_env.state.board.rubble + 10

coords = np.array([start_coord, end_coord])
coords

x_bounds = np.min(coords[:,0]), np.max(coords[:,0])
y_bounds = np.min(coords[:,1]), np.max(coords[:,1])
x_bounds, y_bounds

new_cost = map_cost.copy()
mask = np.ones(map_cost.shape, dtype=int)
mask[y_bounds[0]:y_bounds[1]+1, x_bounds[0]: x_bounds[1]+1] = 0
new_cost[mask == 1] = -1

grid = Grid(matrix=new_cost)
start = grid.node(*start_coord)
end = grid.node(*end_coord)

pathfinder = AStarFinder(diagonal_movement=-1)

In [30]:
%%time
path4, runs = pathfinder.find_path(start, end, grid)
path4

CPU times: total: 0 ns
Wall time: 0 ns


[]

In [18]:
path4 == path1

True

Turn it into a function and also include a bit of a buffer around coords

In [19]:
arr = np.arange(9).reshape(-1, 3)

In [31]:
%%time
start_coord = [1, 1]
end_coord = [1, 1]

def find_path(start_coords, end_coords, cost_map, margin=2):
    coords = np.array([start_coord, end_coord])

    # x, y
    mins = np.min(coords, axis=0) 
    maxs = np.max(coords, axis=0)

    # x, y
    lowers = [max(0, v-margin) for v in mins]
    uppers = [min(s-1, v+margin)+1 for s, v in zip(reversed(cost_map.shape), maxs)]  # +1 for range

    x_range, y_range = [(lowers[i], uppers[i]) for i in range(2)]

    # y, x
    new_cost = cost_map[range(*y_range), :][:, range(*x_range)]

    grid = Grid(matrix=new_cost)
    start = grid.node(*[c-l for c, l in zip(start_coord, lowers)])
    end = grid.node(*[c-l for c, l in zip(end_coord, lowers)])

    pathfinder = AStarFinder(diagonal_movement=-1)
    path, runs = pathfinder.find_path(start, end, grid)
    path = np.array(path)+np.array(lowers)
    return path
    
path = find_path(start_coord, end_coord, map_cost, margin=1)
path

CPU times: total: 0 ns
Wall time: 1 ms


array([[1, 1]])

In [21]:
path

array([[1, 1],
       [1, 2],
       [1, 3],
       [2, 3],
       [3, 3],
       [4, 3],
       [4, 4],
       [5, 4],
       [5, 5]])

In [22]:
arr = np.array([1,2,3])
arr[:3]

array([1, 2, 3])

In [51]:
test_env.state.factories['player_0']['factory_0'].pos.pos, test_env.state.units['player_0']['unit_4'].pos.pos

([23, 37], array([25, 34], dtype=int64))

In [58]:
d = {'a': 1}
d.pop('b')

KeyError: 'b'